v1.1

# Objective

The objective of this notebook is to 
- calcualte the nltk features (nltk_)

# Pre-checks

In [1]:
## Check for Google Drive Connectivity
try:
    from google.colab import drive
    drive.mount('/content/drive')
    google_env = True
except:
    print("Not a Goolge Drive Environment.")
    google_env = False

Not a Goolge Drive Environment.


In [2]:
import time
t_start = time.time()

- The objective of this **01.03** notebook is to 
  - calculate the nltk pos tag features

# Setup Environment

## Install Modules

In [3]:
!pip install watermark
!pip install nltk

In [4]:
import nltk
nltk.download('popular')
nltk.download('stopwords')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/busera/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/busera/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/busera/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/busera/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/busera/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/busera/nltk_data...
[nltk_data]    |   Package movie_reviews is already

True

## Import Modules

In [5]:
# Base libraries
import os
import re
from datetime import date

# Scientific libraries
import numpy as np
import pandas as pd

# Visualization
import seaborn as sns
sns.set(rc={'figure.figsize':(8,4)})
sns.set(font_scale=0.8)

# Helper libraries
from tqdm import tqdm
tqdm.pandas()
from watermark import watermark
import gc # garbage collection to optimize memory usage, use gc.collect()
import warnings
warnings.filterwarnings('ignore')

# Pandas options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Load magic commands
%load_ext watermark

## Define Parameters

In [6]:
seed = 42
cpu_count = os.cpu_count()
cpu_count = cpu_count-2  # to keep machine responsive when fitting the models
notebook_no = "01.03"
today = date.today()

## Global Helper Functions

In [7]:
import http.client
import urllib


def send_push(message):
    """Send push notifications to pushover service."""
    try:
        conn = http.client.HTTPSConnection("api.pushover.net:443")
        conn.request("POST", "/1/messages.json",
                     urllib.parse.urlencode({
                         "token": "ahs1q4mwpnxe3645zeaqzas69whq7a",  # ML Notifications Channel
                         "user": "u5vr1qkc9ghudg2ehuug153okeiz1d",
                         "message": message,
                     }), {"Content-type": "application/x-www-form-urlencoded"})

        conn.getresponse()

    except:
        print("There was a communication issue (pushover).")


# Load Data

In [8]:
# Load Data
if google_env:
    # Location for "shared with" people
    # create a shortcut of the shared folder in your Google Drive root folder
    ROOT_PATH = "/content/drive/MyDrive/SIADS696/Environment/"
else:
    ROOT_PATH = "../"
    print("Not a Google Drive Environment. Loading local files.")

PATH_DATA = "data/"
PATH_DATA_RAW = "data/raw/"
PATH_DATA_INT = "data/interim/"
PATH_DATA_PRO = "data/processed/"
PATH_DATA_MOD = "models/"
PATH_DATA_REP = "reports/"
PATH_DATA_FIG = "reports/figures/"
PATH_DATA_HTML = "reports/html/"

df_wiki_train = pd.read_parquet(ROOT_PATH + PATH_DATA_INT + "train_features_clean_stats.parquet.gzip")
df_wiki_test = pd.read_parquet(ROOT_PATH + PATH_DATA_INT + "test_features_clean_stats.parquet.gzip")


Not a Google Drive Environment. Loading local files.


In [9]:
df_wiki_train.shape
df_wiki_test.shape

(416768, 37)

(119092, 38)

# 4.0 Data Cleaning and Feature Engineering

**Tips on Creating Features**
- Linear models learn sums and differences naturally, but can't learn anything more complex.
- Ratios seem to be difficult for most models to learn. Ratio combinations often lead to some easy performance gains.
- Linear models and neural nets generally do better with normalized features. Neural nets especially need features scaled to values not too far from 0. Tree-based models (like random forests and XGBoost) can sometimes benefit from normalization, but usually much less so.
- Tree models can learn to approximate almost any combination of features, but when a combination is especially important they can still benefit from having it explicitly created, especially when data is limited.
- Counts are especially helpful for tree models, since these models don't have a natural way of aggregating information across many features at once.
[Source](https://www.kaggle.com/code/ryanholbrook/creating-features)

### Count verbs, nouns, and other parts of speech (nltk)

https://practicaldatascience.co.uk/data-science/how-to-use-nltk-for-pos-tagging-in-pandas
https://stackoverflow.com/questions/48930871/creating-a-function-to-count-the-number-of-pos-in-a-pandas-instance

In [10]:
from nltk.corpus import stopwords

In [11]:
def count_nltk_pos_tags(text, nltk_pos_tags):
    a = pd.Series(text)
    result = a.map(lambda x: 1 if x[1]== nltk_pos_tags else 0).sum()

    return result


def count_stopwords(word_tokens):
    nltk_stopwords = (stopwords.words('english'))
    stopwords_list = [w for w in word_tokens if w in nltk_stopwords]

    return len(stopwords_list)


def nltk_pos_tags_count(df):
    nltk_pos_tags = ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ',
                     'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP',
                     'NNPS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 
                     'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 
                     'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 
                     'WP$', 'WRB']

    print("Creating tokens.")
    df['tokenized'] = df['cleaned_text'].progress_apply(nltk.word_tokenize)
    
    print("Counting stopwords.")
    df['nltk_stopwords_count'] = df['tokenized'].progress_apply(count_stopwords)
    
    print("Assigning nltk_pos_tags.")
    df['tagged'] = df['tokenized'].progress_apply(nltk.pos_tag)

    print("Counting nltk_pos_tags.")
    for tag in tqdm(nltk_pos_tags):
        df["nltk_pos_"+tag] = df['tagged'].apply(lambda x: count_nltk_pos_tags(x, tag))

    return df

In [12]:
%%time
# MBPM1: 16.30 min
# Deepnote (Plus): 53 min?
# Windows ML (32 GB): 38 min !!
# Google Colab (Pro): 

df_wiki_train = nltk_pos_tags_count(df_wiki_train)

Creating tokens.


100%|██████████| 416768/416768 [00:21<00:00, 19037.06it/s]


Counting stopwords.


100%|██████████| 416768/416768 [00:26<00:00, 15744.94it/s]


Assigning nltk_pos_tags.


100%|██████████| 416768/416768 [02:43<00:00, 2541.59it/s]


Counting nltk_pos_tags.


100%|██████████| 35/35 [13:44<00:00, 23.56s/it]

CPU times: user 16min 58s, sys: 17.1 s, total: 17min 15s
Wall time: 17min 16s


In [13]:
%%time
# MBPM1: 4.52 min
# Deepnote (Plus): 15 min?
# Google Colab (Pro): 

df_wiki_test = nltk_pos_tags_count(df_wiki_test)

Creating tokens.


100%|██████████| 119092/119092 [00:06<00:00, 19130.04it/s]


Counting stopwords.


100%|██████████| 119092/119092 [00:07<00:00, 15276.98it/s]


Assigning nltk_pos_tags.


100%|██████████| 119092/119092 [00:48<00:00, 2479.35it/s]


Counting nltk_pos_tags.


100%|██████████| 35/35 [03:53<00:00,  6.68s/it]

CPU times: user 4min 50s, sys: 4.98 s, total: 4min 55s
Wall time: 4min 55s


In [14]:
df_wiki_train.head(1).T

,0
original_text,"There is manuscript evidence that Austen continued to work on these pieces as late as the period 1809 â '' 11 , and that her niece and nephew , Anna and James Edward Austen , made further additions as late as 1814 ."
cleaned_text,there is manuscript evidence that austen continued to work on these pieces as late as the period 1809 11 and that her niece and nephew anna and james edward austen made further additions as late as 1814
label,1
stats_lrb_count,0
stats_rrb_count,0
stats_comma_count,3
stats_equalsign_count,0
stats_char_count,173
stats_image_description,0
stats_frac_description,0


In [15]:
df_wiki_test.head(1).T

,0
id,0
cleaned_text,2011
original_text,-2011
label,-1.0
stats_lrb_count,0
stats_rrb_count,0
stats_comma_count,0
stats_equalsign_count,0
stats_char_count,5
stats_image_description,0


# 5.0 Export

In [16]:
columns = df_wiki_train.columns.to_list()
feature_columns = sorted([x for x in columns if x.startswith("nltk_")])

print(len(feature_columns))
print(feature_columns)

36
['nltk_pos_CC', 'nltk_pos_CD', 'nltk_pos_DT', 'nltk_pos_EX', 'nltk_pos_FW', 'nltk_pos_IN', 'nltk_pos_JJ', 'nltk_pos_JJR', 'nltk_pos_JJS', 'nltk_pos_LS', 'nltk_pos_MD', 'nltk_pos_NN', 'nltk_pos_NNP', 'nltk_pos_NNPS', 'nltk_pos_NNS', 'nltk_pos_PDT', 'nltk_pos_POS', 'nltk_pos_PRP', 'nltk_pos_PRP$', 'nltk_pos_RB', 'nltk_pos_RBR', 'nltk_pos_RBS', 'nltk_pos_RP', 'nltk_pos_TO', 'nltk_pos_UH', 'nltk_pos_VB', 'nltk_pos_VBD', 'nltk_pos_VBG', 'nltk_pos_VBN', 'nltk_pos_VBP', 'nltk_pos_VBZ', 'nltk_pos_WDT', 'nltk_pos_WP', 'nltk_pos_WP$', 'nltk_pos_WRB', 'nltk_stopwords_count']


In [17]:
# Export NLTK features
df_export = df_wiki_train[feature_columns]
#df_export.to_csv(ROOT_PATH+PATH_DATA_INT+"train_features_nltk.csv", index=False)
df_export.to_parquet(ROOT_PATH+PATH_DATA_INT+"train_features_nltk.parquet.gzip", compression='gzip')

df_export = df_wiki_test[feature_columns]
#df_export.to_csv(ROOT_PATH+PATH_DATA_INT+"test_features_nltk.csv", index=False)
df_export.to_parquet(ROOT_PATH+PATH_DATA_INT+"test_features_nltk.parquet.gzip", compression='gzip')


# 6.0 Watermark

In [18]:
%watermark

Last updated: 2023-02-26T17:52:13.517981+01:00

Python implementation: CPython
Python version       : 3.9.0
IPython version      : 8.9.0

Compiler    : Clang 11.0.0 
OS          : Darwin
Release     : 22.3.0
Machine     : arm64
Processor   : arm
CPU cores   : 10
Architecture: 64bit



In [19]:
%watermark --iversions

pandas : 1.5.3
re     : 2.2.1
nltk   : 3.8.1
seaborn: 0.12.2
numpy  : 1.23.5



In [20]:
t_end = time.time()
total_runtime = t_end-t_start
total_runtime_min = round((total_runtime/60),2)
print(str(total_runtime_min)+" minutes")

22.32 minutes


In [21]:
send_push(f"01.03 Feature Engineering: Calculate nltk features finished in: {total_runtime_min} min.")

In [22]:
output_file = f'{ROOT_PATH}{PATH_DATA_HTML}{today}_01.03_calculate_nltk_features.html'
!jupyter nbconvert --to html "01.03_calculate_nltk_features.ipynb" --output {output_file}

NameError: name 'today' is not defined